In [1]:
import pandas as pd
import datetime as dt
from datetime import *
import numpy as np

In [2]:
cf = pd.read_csv('classification.csv')

In [3]:
df = pd.read_csv('b.csv')
df = df[(df.METRO == 'Nonmetropolitan') &  (df.FULLPART != 'Part time')].copy()
#df = df[(df.METRO == 'Nonmetropolitan') &  (df.FULLPART == 'Full time') & (df.ACTIVITY == 'Work, main job')]
df['START'] = pd.to_datetime(df['START'])

df['STOP'] = pd.to_datetime(df['STOP'])

df = df.merge(cf, left_on='ACTIVITY', right_on='activity').copy()
df.head()

,RECTYPE,CASEID,METRO,FULLPART,ACTLINE,ACTIVITY,START,STOP,activity,location
0,3,20170101170090,Nonmetropolitan,Full time,1.0,Travel related to working,2019-05-20 04:00:00,2019-05-20 05:45:00,Travel related to working,P
1,3,20170101170090,Nonmetropolitan,Full time,3.0,Travel related to working,2019-05-20 14:00:00,2019-05-20 15:45:00,Travel related to working,P
2,3,20170101170572,Nonmetropolitan,Full time,6.0,Travel related to working,2019-05-20 11:30:00,2019-05-20 11:45:00,Travel related to working,P
3,3,20170101170572,Nonmetropolitan,Full time,13.0,Travel related to working,2019-05-20 21:00:00,2019-05-20 21:15:00,Travel related to working,P
4,3,20170101170726,Nonmetropolitan,Full time,13.0,Travel related to working,2019-05-20 16:26:00,2019-05-20 16:46:00,Travel related to working,P


In [4]:
interval = 30*60
midnight  = datetime.combine(datetime.today(), datetime.min.time())
end_day = datetime.combine(datetime.today(), datetime.max.time())

def a(s):
    if s.START > s.STOP:
        r = (end_day - datetime.combine(datetime.today(), s.START.time())) + (s.STOP- midnight)       
    else:
        r = s.STOP - s.START
    return r
        

df['duration'] = df.apply(a,axis=1)


In [5]:
def normalize(m):
    row_sums = m.sum(axis=1)
    return m / row_sums[:, np.newaxis]

In [7]:
t1 = datetime.combine(datetime.today(), time(19,0,0))
t2 = datetime.combine(datetime.today(), time(6,30,0))
t3 = datetime.combine(datetime.today(), time(9,30,0))
t4 = datetime.combine(datetime.today(), time(16,30,0))


# change this to != 'Full time' for unemployed transition matrix
df = df[(df.FULLPART == 'Full time')].copy()

In [8]:
interval = 30*60
A_times = [(x-interval, x-1) for x in range(int(19*60*60), (int(24*60*60)), interval)[1:]] + \
           [(x-interval, x-1) for x in range(0, (int(6.5*60*60)), interval)[1:]]
B_times = [(x-interval, x-1) for x in range(int(6.5*60*60), (int(9.5*60*60))+interval, interval)[1:]]
C_times = [(x-interval, x-1) for x in range(int(16.5*60*60), (int(19*60*60)), interval)[1:]]
D_times = [(x-interval, x-1) for x in range(int(9.5*60*60), (int(16.5*60*60)), interval)[1:]]

#A_times, B_times, C_times, D_times

In [9]:
def get_transit_matrix(m, df, times):
    midnight  = datetime.combine(datetime.today(), datetime.min.time())
    k = {'H':0, 'W':1, 'P':2}
    
    
    for case in df.CASEID.unique(): #3[20170101170090]:#
       
        b = df[df.CASEID == case].copy()
        c_loc = None
        ldd = set()
        
        status = np.zeros((3, 3),dtype=int)
        
        
        for start,end in times:
            
            start_time = (midnight + timedelta(seconds= start))
            end_time = (midnight + timedelta(seconds=end))
        
            nature_mask = (b.START > b.STOP)
            true_mask = (b.STOP > start_time) | (b.START < end_time)
            false_mask = (b.START < end_time) & (b.STOP > start_time)
            
            
            #print(start_time, end_time)

          
            final =  ((nature_mask & true_mask) | (~nature_mask & false_mask))
            #print(false)

            b = b[final].copy()
            r = b.sort_values(['ACTLINE'], ascending=False)


            if len(r.location)>0:
                loc = list(r.location)
                
                if c_loc == None:
                    c_loc = loc[0]
                    loc = loc[1:]
                    ldd.add(c_loc)
            
                for l in loc:
                    ldd.add(l)
                    if c_loc  != l and status[k[c_loc]][k[l]] == 0:

                        m[k[c_loc]][k[l]] +=1
                        status[k[c_loc]][k[l]] += 1
                    #c_loc = l
                    c_loc = l
                    
        
        t = 0
        for i in range(len(status)):
            t += sum(status[i])
            
        if t == 0 and c_loc != None:
            m[k[c_loc]][k[c_loc]] +=1
            
            if len(ldd) > 1:
                print(len(ldd))
            
            #pass
        
               # c_loc = l

    return m

In [10]:
A = np.zeros((3, 3),dtype=int)
B = np.zeros((3, 3),dtype=int)
C = np.zeros((3, 3),dtype=int)
D = np.zeros((3, 3),dtype=int)


for tb in [1, 2, 3, 4]:
    if tb ==1:
        A = get_transit_matrix(A, df, A_times)
        print('A is done!')
        
    elif (tb == 2):
        B = get_transit_matrix(B, df, B_times)
        print('B is done!')
    
    elif (tb == 3):
        C = get_transit_matrix(C, df, C_times)
        print('C is done!')
                
    elif (tb == 4):
        D = get_transit_matrix(D, df, D_times)
        print('D is done!')

A is done!
B is done!
C is done!
D is done!


In [11]:
A

array([[410,   9,  61],
       [ 10,  57,   3],
       [ 60,   5,  89]])

In [12]:
B

array([[435,  40,  31],
       [ 23,  83,  29],
       [ 42,  15,  26]])

In [13]:
C

array([[270,   9,  58],
       [  7, 146,  13],
       [ 58,  12, 136]])

In [14]:
D

array([[241,  25,  35],
       [ 17, 243,  10],
       [ 37,   7,  79]])

{'H':0, 'W':1, 'P':2}

In [15]:
normalize(A)

array([[0.85416667, 0.01875   , 0.12708333],
       [0.14285714, 0.81428571, 0.04285714],
       [0.38961039, 0.03246753, 0.57792208]])

In [16]:
normalize(B)

array([[0.85968379, 0.07905138, 0.06126482],
       [0.17037037, 0.61481481, 0.21481481],
       [0.5060241 , 0.18072289, 0.31325301]])

In [17]:
normalize(C)

array([[0.80118694, 0.02670623, 0.17210682],
       [0.04216867, 0.87951807, 0.07831325],
       [0.2815534 , 0.05825243, 0.66019417]])

In [18]:
normalize(D)

array([[0.80066445, 0.08305648, 0.11627907],
       [0.06296296, 0.9       , 0.03703704],
       [0.30081301, 0.05691057, 0.64227642]])